In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('connection_data//Connections_Data.csv')

In [2]:
# pip install gensim
import gensim.downloader as api

path = api.load("fasttext-wiki-news-subwords-300", return_path=True)
print("fastText file downloaded at", path)

fastText file downloaded at C:\Users\admin/gensim-data\fasttext-wiki-news-subwords-300\fasttext-wiki-news-subwords-300.gz


In [3]:
import gzip
import shutil
from pathlib import Path

gz_path = Path(path)
vec_path = gz_path.with_suffix(".vec")

if vec_path.exists():
    print(f"Vec file already exists, skipping unzip: {vec_path}")
else:
    print("Start unzipping...")
    with gzip.open(gz_path, 'rb') as f_in:
        with open(vec_path, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
    print(f"Unzipping complete: {vec_path}")

Vec file already exists, skipping unzip: C:\Users\admin\gensim-data\fasttext-wiki-news-subwords-300\fasttext-wiki-news-subwords-300.vec


In [4]:
raw_words = df["Word"].dropna()
words = set(w.strip().lower().replace(" ", "-") for w in raw_words)

vec_path = Path(vec_path)

model = {}
vector_dim = None

print("Start scanning .vec file (this may take a moment)...")

with open(vec_path, "r", encoding="utf-8", errors="ignore") as f:
    # read header
    header = f.readline().strip().split()
    if len(header) == 2 and header[0].isdigit():
        vector_dim = int(header[1])
    else:
        f.seek(0)

    for line in f:
        parts = line.rstrip().split()
        if len(parts) < 10:
            continue

        word = parts[0].lower()
        if word in words:
            vec = np.asarray(parts[1:], dtype=np.float32)
            if vector_dim is None:
                vector_dim = vec.shape[0]

            model[word] = vec

            if len(model) == len(words):
                break

print(f"Found {len(model)} / {len(words)} words.")
print("Done.")

Start scanning .vec file (this may take a moment)...
Found 6550 / 6816 words.
Done.


In [5]:
model['ice-cube']

array([-0.017448  , -0.006353  , -0.043742  ,  0.028487  , -0.048163  ,
       -0.020148  , -0.028998  , -0.059167  , -0.016275  ,  0.002336  ,
        0.039239  , -0.014074  ,  0.0049962 , -0.031001  ,  0.012295  ,
       -0.0081254 ,  0.035317  ,  0.047976  ,  0.11168   ,  0.01439   ,
        0.012294  ,  0.041442  , -0.016208  ,  0.045247  ,  0.050349  ,
       -0.026719  , -0.010164  ,  0.0029119 ,  0.046074  ,  0.00019521,
       -0.043871  , -0.028271  , -0.0066389 ,  0.02865   ,  0.042338  ,
       -0.024059  , -0.0066317 , -0.018532  ,  0.078987  , -0.010295  ,
        0.0048011 , -0.13805   , -0.037614  ,  0.019327  ,  0.022345  ,
       -0.006914  , -0.0098834 , -0.020254  , -0.0025689 , -0.0044975 ,
       -0.028775  ,  0.0022331 ,  0.0004047 ,  0.016868  , -0.030643  ,
        0.0072221 ,  0.023955  , -0.044592  , -0.044961  , -0.053063  ,
        0.0019819 , -0.0015742 ,  0.081193  , -0.035179  , -0.012067  ,
        0.028345  ,  0.010307  , -0.032199  , -0.037867  , -0.00